### Exercise 2: Real-time Data Analysis and Feedback on Live Acquisition ###

"""
This script simulates a real-time data analysis and feedback system on a live acquisition setup. Using the MicroManager
simulated "Noise" mode, it performs a timelapse acquisition in the DAPI channel. Each acquired image is inspected for pixels
with a value of exactly 700 counts. For such pixels, neighboring pixels within a 15-pixel radius are set to zero. The
modified images are then saved to a .tif dataset.

This example illustrates the application of real-time image processing principles to lab systems, despite being a
simulated environment.

Requirements:
- MicroManager in "Noise" mode for the camera.
- Timelapse acquisition of 100 time points with 10 ms exposure time.
- Real-time pixel manipulation based on specific criteria.

Acknowledgments:
- Use of scikit-image for drawing disk and manipulation was informed by scikit-image documentation.
- The image processing technique was conceptualized by micro-manager documentation and discussion forum (https://forum.image.sc/), with the assistance of ChatGPT.
"""

In [2]:
# Step 1: Import necessary libraries
!pip install scikit-image
from pycromanager import Acquisition, multi_d_acquisition_events, Core
import numpy as np
from skimage import draw

# Initialize MicroManager Core
core = Core()

Exception: Couldn't create Core. Is Micro-Manager running and is the ZMQ server on 4827 option enabled?

In [2]:
# Step 2: Set acquisition parameters
num_time_points = 100   # Total number of time points for timelapse
exposure_time_ms = 10   # Exposure time in milliseconds
channels = ['DAPI']     # Acquisition channel

In [3]:
# Specify the directory for saving the data
save_directory = 'C:/Users/chang/Downloads/2024_AssocRDEng_TakeHome/Exercise2'
acquisition_name = 'my_acquisition'    

In [4]:
# Step 3: Generate multi-dimensional acquisition events
# Note: This acquisition setup does not require adjustments to the x and y positions.
## Same inspiration in Exercise1: multi_d_acquisition_events() in pycromanager official document

events = multi_d_acquisition_events(
    num_time_points=num_time_points,
    channels=channels,
    channel_exposures_ms=[exposure_time_ms],
)

In [1]:
'''
Acknowledgment: 
# The image_process_fn() was inspired by official document: https://pycro-manager.readthedocs.io/en/latest/apis.html
# The use of call- out definition function instead of inserting in Acqusition was inspired by (https://pycro-manager.readthedocs.io/en/latest/apis.html) and (https://pycro-manager.readthedocs.io/en/latest/img_processors.html)
# The use of scikit-image for drawing disk and manipulation was informed by scikit-image documentation.

'''

# Step 4: Define the image processing function
def process_image(image):
    """Inspect and modify the image based on specific pixel values."""
    print("Original image shape:", image.shape)
    
    # Find pixels with exactly 700 counts
    position700 = np.column_stack(np.where(image == 700))
    print(position700)         # To see whether the position "image=700" exists in the random acquisition of the noise
    for y, x in position700:
        y, x = int(y), int(x)

        # Set neighboring pixels within a radius of 15 to zero
        ## rr, cc = draw.circle(y, x, 15, shape=image.shape) ## draw.corcle() is an old function that cannot be fit
        rr, cc = draw.disk((y, x), 15, shape=image.shape)
        image[rr.astype(int), cc.astype(int)] = 0
        
        print(image) # Print out to check that the shape and type of image are correct
        
    return image


def image_processor_fn(image, metadata, event):
    """Process each acquired image in real-time."""
    # Print out to ensure this function has been correctly executed and  'image' and 'event' are the correct data types
    print("The type of image", type(image))
    print("The type of event", type(metadata))

    modified_image = process_image(image)
    print("Finished one of the Images!")
    
    # Return to modified image and event(metadata)
    return modified_image, metadata


# Step 5: Perform the acquisition and real-time image processing
## Using Acquisition to do image acquisition, and send image into image_process_fn()
with Acquisition(directory=save_directory, name=acquisition_name,image_process_fn=image_processor_fn) as acq:
    for i, event in enumerate(events):
        try:
            acq.acquire(event)  
            print(f"Acquisition for event {i} done.") 
        except Exception as e:
            print(f"Error during acquisition for event {i}: {e}") 
    

# Step 6: Confirm completion
print("All IMAGES ARE FINISHED!")
print(f"The images have been processed and saved to {save_directory}/{acquisition_name}")


NameError: name 'Acquisition' is not defined